# Missing values

1. This notebook explores the approaches to handle missing values
2. INSEAD Chiller Plant data is first considered. However, North Point would be ideal (let's probably use it as well)

In [1]:
% matplotlib inline 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

In [136]:
# load data
def load_df(src, pattern):
    all_files = glob.glob(os.path.join(src, pattern))
    df = pd.concat([pd.read_csv(f, low_memory=False) for f in all_files], ignore_index=True)

    # minor changes
    df = df.rename(columns={"Time Stamp": "timestamp"})
    df = df.replace("\\N", np.nan)

    # update data types. object is taken as default
    dtypes = dict([(col, np.float64) for col in df.columns])
    dtypes["timestamp"] = "datetime64[ns]"
    df = df.astype(dtypes)

    # change the index to timestamp.
    df.index = df.timestamp
    
    return df

# load all csv files of North-point site. Concatenate them..
data_src = "../data/insead"
pattern = "*21*.csv" # don't want to load all data (500MB or more!)
df = load_df(data_src, pattern)

In [45]:
(df < 0).sum() * 100 / df.count()

timestamp       0.000000
chwrhdr         0.000000
chwshdr         0.015236
chwsfhdr        0.000000
cwshdr          0.000000
cwrhdr          0.000000
cwsfhdr         0.029087
ch1kw           0.000000
ch2kw           0.000000
ch3kw           0.000000
chwp1kw         0.000000
chwp2kw         0.000000
chwp3kw         2.156584
chwp4kw         9.131117
cwp1kw          0.000346
cwp2kw          0.000000
cwp3kw         44.370976
ct1kw           0.000000
ct2akw          0.000000
incomingkw1     0.000000
incomingkw2     0.000000
rh              0.000693
drybulb         0.000000
chiller_eff     0.361162
cwp_eff         0.361162
chwp_eff        0.361162
ct_eff          0.361162
chwgpmrt        0.356661
cwgpmrt         0.363932
systotpower     0.000000
loadsys         0.354929
effsys          0.358046
hbsys          29.694241
dtype: float64

In [46]:
df.isnull().sum() * 100 / df.count()

timestamp      0.000000
chwrhdr        0.000000
chwshdr        0.000000
chwsfhdr       0.000000
cwshdr         0.000000
cwrhdr         0.000000
cwsfhdr        0.000000
ch1kw          0.000000
ch2kw          0.000000
ch3kw          0.000000
chwp1kw        0.000000
chwp2kw        0.000000
chwp3kw        0.000000
chwp4kw        0.021820
cwp1kw         0.000000
cwp2kw         0.000000
cwp3kw         0.501131
ct1kw          0.000000
ct2akw         0.010043
incomingkw1    0.021820
incomingkw2    0.021820
rh             0.021127
drybulb        0.021127
chiller_eff    0.000000
cwp_eff        0.000000
chwp_eff       0.000000
ct_eff         0.000000
chwgpmrt       0.000000
cwgpmrt        0.000000
systotpower    0.000000
loadsys        0.000000
effsys         0.000000
hbsys          0.000000
dtype: float64

In [137]:
# step 0: remove negative values from the columns
def remove_negatives(df, columns=[]):
    columns = columns or df.columns
    for col in columns:
        if col != "timestamp":
            df.loc[df[col] < 0, col] = 0
    return df

df = remove_negatives(df)

In [138]:
# Need to identify values when plant is possibly shutdown..
# Look at CT1KW < 0.1.. Is it realistic that plant is working at 0.1kw?
print(df.ct1kw.min(), df.ct1kw.max(), df.ct1kw.median())
print(((df.ct1kw > 0) & (df.ct1kw < 0.1)).sum())
print(((df.ct2akw > 0) & (df.ct2akw < 0.1)).sum())

def identify_ct_shutdown(df, ctkw_cols=[], thresh=0.1):
    ctkw_cols = ctkw_cols or [i for i in df.columns if i.startswith("ct") and i.endswith("kw")]
    for col in ctkw_cols:
        df[col + "_shutdown"] = 0
        df.loc[df[col] < thresh, col + "_shutdown"] = 1
    return df


# replace ctkw < thresh values with 0
def replace_ct_values(df, ctkw_cols=[], thresh=0.1):
    ctkw_cols = ctkw_cols or [i for i in df.columns if i.startswith("ct") and i.endswith("kw")]
    for col in ctkw_cols:
        df.loc[df[col] < thresh, col] = 0
    return df

df = identify_ct_shutdown(df)
df = replace_ct_values(df)


print(((df.ct1kw > 0) & (df.ct1kw < 0.1)).sum())
print(((df.ct2akw > 0) & (df.ct2akw < 0.1)).sum())

0.0 24.832 4.672
62905
74667
0
0


In [ ]:
# Handling Null data..

# Take rolling average. Doesn't cover all data if the sequence is looong
# We don't want to have a big window either.. (might pollute data)
def missing_approach1(df, columns)
    df[columns] = df[columns].rolling(5, min_periods=1).mean()
    return df

# Look back. Replace the value with previous day's
# if the null sequence is too long, we'd get a flat signal. Possibly a 
def missing_approach2(df, columns, lookback=3):
    pass